In [390]:
import pandas as pd
import numpy as np
from datetime import timedelta

Read in data

In [391]:
w_res = pd.read_csv('6km_results.csv')

In [392]:
m_res = pd.read_csv('10km_results.csv')

Add columns
* Team position - rank within team (ex. a team's fifth runner will have a `team_pos` value of 5).
* Flight score - rank among all runners with the same team position (ex. how do 4th runners from all teams compare to each other)

In [393]:
w_res['team_pos'] = w_res[w_res['points']>0].groupby('school')['points'].rank(ascending=True, method='dense')
w_res['flight_score'] = w_res.groupby('team_pos')['points'].rank(ascending=True, method='dense')
w_res['time_in_seconds'] = pd.to_timedelta('00:' + w_res['time']).dt.total_seconds()

In [394]:
m_res['team_pos'] = m_res[m_res['points']>0].groupby('school')['time'].rank(ascending=True, method='dense')
m_res['flight_score'] = m_res.groupby('team_pos')['points'].rank(ascending=True, method='dense')
m_res['time_in_seconds'] = pd.to_timedelta('00:' + m_res['time']).dt.total_seconds()

Fill in values for runners who DNF
* Will just fill in the maximum values for `position`, `points`, and `team_pos` for runners that did not finish.

In [395]:
w_res['position'] = np.where(w_res['time'].isna(), w_res['position'].max()+1, w_res['position'])
w_res['points'] = np.where(w_res['time'].isna(), w_res['points'].max()+1, w_res['points'])
w_res['team_pos'] = np.where(w_res['time'].isna(), w_res['team_pos'].max(), w_res['team_pos'])


In [396]:
m_res['position'] = np.where(m_res['time'].isna(), m_res['position'].max()+1, m_res['position'])
m_res['points'] = np.where(m_res['time'].isna(), m_res['points'].max()+1, m_res['points'])
m_res['team_pos'] = np.where(m_res['time'].isna(), m_res['team_pos'].max(), m_res['team_pos'])


What would the podium look like if we scored 6 runners? 7 runners?

In [397]:
for i in range(4,8):
    print(i)
    scores_temp = w_res[w_res['team_pos']<=i].groupby('school').agg({'points':'sum', 'name':'count'})
    scores_temp = scores_temp[scores_temp['name']==i]
    scores_temp['team_place'] = scores_temp['points'].rank(ascending=True, method='dense')
    scores_temp['n_runners'] = i
    scores_temp['point_diff'] = scores_temp.sort_values('team_place')['points'] - scores_temp.sort_values('team_place')['points'].shift(1)
    if i==4:
       scores = scores_temp
    else:
       scores = pd.concat([scores,scores_temp])

4
5
6
7


In [398]:
scores[scores['team_place']<=4].sort_values(['n_runners','team_place'])

,points,name,team_place,n_runners,point_diff
school,,,,,
Providence,85,4,1.0,4,NaN
West Virginia,97,4,2.0,4,12.0
BYU,98,4,3.0,4,1.0
Stanford,122,4,4.0,4,24.0
BYU,147,5,1.0,5,NaN
West Virginia,164,5,2.0,5,17.0
Providence,183,5,3.0,5,19.0
Northern Arizona,206,5,4.0,5,23.0
BYU,209,6,1.0,6,NaN


In [399]:
six_runners_w = scores[scores['n_runners']==6].sort_values(['team_place'])[['team_place','points']]
seven_runners_w = scores[scores['n_runners']==7].sort_values(['team_place'])[['team_place','points']]

six_runners_w.to_csv('six_runners_w.csv',header=True)
seven_runners_w.to_csv('seven_runners_w.csv',header=True)


In [400]:
for i in range(4,8):
    scores_temp = m_res[m_res['team_pos']<=i].groupby('school').agg({'points':'sum', 'name':'count'})
    scores_temp = scores_temp[scores_temp['name']==i]
    scores_temp['team_place'] = scores_temp['points'].rank(ascending=True, method='dense')
    scores_temp['n_runners'] = i
    scores_temp['point_diff'] = scores_temp.sort_values('team_place')['points'] - scores_temp.sort_values('team_place')['points'].shift(1)
    if i==4:
        scores = scores_temp
    else:
        scores = pd.concat([scores,scores_temp])

In [401]:
scores[scores['team_place']<=4].sort_values(['n_runners','team_place'])

,points,name,team_place,n_runners,point_diff
school,,,,,
BYU,81,4,1.0,4,NaN
Arkansas,91,4,2.0,4,10.0
Iowa State,95,4,3.0,4,4.0
North Carolina,131,4,4.0,4,36.0
BYU,124,5,1.0,5,NaN
Iowa State,137,5,2.0,5,13.0
Arkansas,202,5,3.0,5,65.0
Wisconsin,212,5,4.0,5,10.0
BYU,178,6,1.0,6,NaN


In [402]:
six_runners_m = scores[scores['n_runners']==6].sort_values(['team_place'])[['team_place','points']]
seven_runners_m = scores[scores['n_runners']==7].sort_values(['team_place'])[['team_place','points']]

six_runners_m.to_csv('six_runners_m.csv',header=True)
seven_runners_m.to_csv('seven_runners_w.csv',header=True)

Flight scoring - rank all the 1st runners, all the 2nd runners, etc. and add the scores together

In [403]:
flight_score_w_res = w_res[w_res['team_pos']<=5].groupby('school').agg({'flight_score':'sum', 'name':'count'})
flight_score_w_res = flight_score_w_res[flight_score_w_res['name']==5]
flight_score_w_res['team_place'] = flight_score_w_res['flight_score'].rank(ascending=True, method='dense')
flight_score_w_res.sort_values('team_place').head(4)

,flight_score,name,team_place
school,,,
West Virginia,18.0,5,1.0
BYU,22.0,5,2.0
Providence,23.0,5,3.0
Stanford,28.0,5,4.0


In [404]:
flight_score_w_res.sort_values('team_place')[['team_place','flight_score']].to_csv('flight_score_w_res.csv', header=True)

In [405]:
flight_score_m_res = m_res[m_res['team_pos']<=5].groupby('school').agg({'flight_score':'sum', 'name':'count'})
flight_score_m_res = flight_score_m_res[flight_score_m_res['name']==5]
flight_score_m_res['team_place'] = flight_score_m_res['flight_score'].rank(ascending=True, method='dense')
flight_score_m_res.sort_values('team_place').head(4)

,flight_score,name,team_place
school,,,
BYU,15.0,5,1.0
Iowa State,20.0,5,2.0
Arkansas,22.0,5,3.0
North Carolina,31.0,5,4.0


In [406]:
flight_score_m_res.sort_values('team_place')[['team_place','flight_score']].to_csv('flight_score_m_res.csv', header=True)

Runners that finished top amongst their "peer group"

In [407]:
w_res[w_res['flight_score']==1]

,position,name,school,year,time,split,position_change_in_split,points,team_pos,flight_score,time_in_seconds
0,1,Doris Lemngole,Alabama,SO,19:21.0,2:55.8,1,1,1.0,1.0,1161.0
7,8,Hannah Gapes,NC State,SO,19:42.7,3:10.4 | +21.7,3,8,2.0,1.0,1182.7
23,24,Shannon Flockhart,Providence,SR,19:54.2,3:12.9 | +33.2,3,19,3.0,1.0,1194.2
42,43,Taylor Rohatinsky,BYU,SO,20:06.5,3:16.3 | +45.5,2,33,4.0,1.0,1206.5
64,65,Carlee Hansen,BYU,SR,20:21.6,3:20.7 | +1:00.7,6,49,5.0,1.0,1221.6
82,83,Nelah Roberts,BYU,FR,20:27.4,3:17.4 | +1:06.4,18,62,6.0,1.0,1227.4
87,88,Taylor Lovell,BYU,JR,20:28.6,3:18.3 | +1:07.6,15,66,7.0,1.0,1228.6


In [408]:
m_res[m_res['flight_score']==1]

,position,name,school,year,time,split,position_change_in_split,points,team_pos,flight_score,time_in_seconds
0,1,Graham Blanks,Harvard,SR,28:37.2,2:45.3,-1,1,1.0,1.0,1717.2
7,8,Ethan Strand,North Carolina,SR,28:53.0,2:50.9 | +15.9,-1,8,2.0,1.0,1733.0
20,21,Patrick Kiprop,Arkansas,SR,29:15.7,2:58.8 | +38.6,6,20,3.0,1.0,1755.7
36,37,Robin Kwemoi Bera,Iowa State,FR,29:23.9,2:55.3 | +46.7,8,33,4.0,1.0,1763.9
48,49,Gable Sieperda,Iowa State,SR,29:31.7,2:51.7 | +54.5,9,42,5.0,1.0,1771.7
61,62,James Corrigan,BYU,SO,29:38.4,2:53.3 | +1:01.3,6,54,6.0,1.0,1778.4
78,79,Aidan Troutner,BYU,SR,29:46.6,3:07.3 | +1:09.5,26,69,7.0,1.0,1786.6


In [409]:
w_res[w_res['school']=='Stanford']

,position,name,school,year,time,split,position_change_in_split,points,team_pos,flight_score,time_in_seconds
3,4,Amy Bunnage,Stanford,SO,19:31.1,3:05.3 | +10.1,-1,4,1.0,4.0,1171.1
32,33,Sophia Kennedy,Stanford,SO,20:00.6,3:13.4 | +39.6,5,25,2.0,7.0,1200.6
57,58,Riley Stewart,Stanford,JR,20:18.1,3:15.5 | +57.1,17,43,3.0,7.0,1218.1
65,66,Zofia Dudek,Stanford,SR,20:21.7,3:24.6 | +1:00.7,4,50,4.0,3.0,1221.7
118,119,Julia Flynn,Stanford,SO,20:40.2,3:20.6 | +1:19.2,16,91,5.0,7.0,1240.2
198,199,Nicola Hogg,Stanford,JR,21:10.3,3:31.1 | +1:49.3,2,161,6.0,15.0,1270.3
223,224,Audrey DaDamio,Stanford,JR,21:25.5,3:30.3 | +2:04.5,6,186,7.0,15.0,1285.5


In [410]:
m_res[m_res['school']=='North Carolina']

,position,name,school,year,time,split,position_change_in_split,points,team_pos,flight_score,time_in_seconds
6,7,Parker Wolfe,North Carolina,SR,28:50.2,2:53.3 | +13.1,2,7,1.0,7.0,1730.2
7,8,Ethan Strand,North Carolina,SR,28:53.0,2:50.9 | +15.9,-1,8,2.0,1.0,1733.0
51,52,Colton Sands,North Carolina,JR,29:34.1,2:54.2 | +57.0,4,45,3.0,5.0,1774.1
81,82,Will Coogan,North Carolina,JR,29:47.9,2:57.9 | +1:10.7,5,71,4.0,7.0,1787.9
141,142,Patrick Anderson,North Carolina,JR,30:11.5,2:56.8 | +1:34.4,8,115,5.0,11.0,1811.5
170,171,Joe Sapone,North Carolina,FR,30:27.3,2:56.2 | +1:50.2,6,141,6.0,11.0,1827.3
206,207,Jacob Laney,North Carolina,FR,30:55.2,3:06.8 | +2:18.1,5,172,7.0,9.0,1855.2


Which team had smallest 1-5 spread?

Time

In [411]:
w_spread = w_res[(w_res['points']>0) & (w_res['team_pos']<=5)].groupby('school').agg({'time_in_seconds':['min','max','count']})
w_spread['spread'] = (w_spread[('time_in_seconds','max')] - w_spread[('time_in_seconds','min')])
w_spread['team_place'] = w_spread['spread'].rank(ascending=True, method='dense')
w_spread.sort_values('team_place').head(5)

time_in_seconds               spread team_place
                           min     max count                  
school                                                        
Wisconsin               1236.6  1251.8     5   15.2        1.0
Washington              1218.3  1245.3     5   27.0        2.0
Boston College          1246.6  1276.6     5   30.0        3.0
Syracuse                1232.1  1262.8     5   30.7        4.0
BYU                     1188.4  1221.6     5   33.2        5.0

In [412]:
w_spread.sort_values('team_place')[['team_place','spread']].to_csv('w_spread_time.csv', header=True)

In [413]:
m_spread = m_res[(m_res['points']>0) & (m_res['team_pos']<=5)].groupby('school').agg({'time_in_seconds':['min','max','count']})
m_spread['spread'] = (m_spread[('time_in_seconds','max')] - m_spread[('time_in_seconds','min')])
m_spread['team_place'] = m_spread['spread'].rank(ascending=True, method='dense')
m_spread.sort_values('team_place').head(4)

time_in_seconds               spread team_place
                        min     max count                  
school                                                     
Stanford             1778.7  1798.9     5   20.2        1.0
Wisconsin            1749.2  1780.5     5   31.3        2.0
Iowa State           1739.8  1771.7     5   31.9        3.0
Wake Forest          1761.6  1795.7     5   34.1        4.0

In [414]:
m_spread.sort_values('team_place')[['team_place','spread']].to_csv('m_spread_time.csv', header=True)

Place

In [415]:
w_spread_place = w_res[(w_res['points']>0) & (w_res['team_pos']<=5)].groupby('school').agg({'position':['min','max','count']})
w_spread_place['spread'] = (w_spread_place[('position','max')] - w_spread_place[('position','min')])
w_spread_place['team_place'] = w_spread_place['spread'].rank(ascending=True, method='dense')
w_spread_place.sort_values('team_place').head(4)

position            spread team_place
                      min  max count                  
school                                                
Wisconsin             111  158     5     47        1.0
BYU                    14   65     5     51        2.0
Northern Arizona       22   84     5     62        3.0
Boston College        143  211     5     68        4.0

In [416]:
w_spread_place.sort_values('team_place').droplevel(level=1, axis=1).to_csv('w_spread_place.csv', header=True)

In [417]:
m_spread_place = m_res[(m_res['points']>0) & (m_res['team_pos']<=5)].groupby('school').agg({'position':['min','max','count']})
m_spread_place['spread'] = (m_spread_place[('position','max')] - m_spread_place[('position','min')])
m_spread_place['team_place'] = m_spread_place['spread'].rank(ascending=True, method='dense')
m_spread_place.sort_values('team_place').head(4)

position            spread team_place
                min  max count                  
school                                          
Iowa State       10   49     5     39        1.0
BYU               6   50     5     44        2.0
Stanford         63  112     5     49        3.0
Wisconsin        16   69     5     53        4.0

In [418]:
m_spread_place.sort_values('team_place').droplevel(level=1, axis=1).to_csv('m_spread_place.csv', header=True)

Time instead of place

In [419]:
time_scoring_m_res = m_res[m_res['team_pos']<=5].groupby('school').agg({'time_in_seconds':'sum', 'name':'count'})
time_scoring_m_res[time_scoring_m_res['name']==5]
time_scoring_m_res['team_place'] = time_scoring_m_res['time_in_seconds'].rank(ascending=True, method='dense')
time_scoring_m_res['overall_time'] = time_scoring_m_res['time_in_seconds'].apply(lambda x: str(timedelta(seconds=round(x))))
time_scoring_m_res.sort_values('team_place').head(4)

,time_in_seconds,name,team_place,overall_time
school,,,,
BYU,8766.6,5,1.0,2:26:07
Iowa State,8794.6,5,2.0,2:26:35
Arkansas,8813.8,5,3.0,2:26:54
North Carolina,8836.7,5,4.0,2:27:17


In [420]:
time_scoring_m_res.sort_values('team_place')[['team_place','overall_time']].to_csv('time_scoring_m_res.csv', header=True)

In [421]:
time_scoring_w_res = w_res[w_res['team_pos']<=5].groupby('school').agg({'time_in_seconds':'sum', 'name':'count'})
time_scoring_w_res[time_scoring_w_res['name']==5]
time_scoring_w_res['team_place'] = time_scoring_w_res['time_in_seconds'].rank(ascending=True, method='dense')
time_scoring_w_res['overall_time'] = time_scoring_w_res['time_in_seconds'].apply(lambda x: str(timedelta(seconds=round(x))))
time_scoring_w_res.sort_values('team_place').head(4)

,time_in_seconds,name,team_place,overall_time
school,,,,
BYU,6019.5,5,1.0,1:40:20
West Virginia,6024.1,5,2.0,1:40:24
Providence,6030.8,5,3.0,1:40:31
Oregon,6049.0,5,4.0,1:40:49


In [422]:
time_scoring_w_res.sort_values('team_place')[['team_place','overall_time']].to_csv('time_scoring_w_res.csv', header=True)